# CDR: High Level Turbine Design

> Author: Elias Aoubala

> Date: 21/04/2025

In [1]:
from turborocket.meanline.meanline_relations import TurbineStageDesign
from turborocket.fluids.fluids import IdealGas
import numpy as np
import pandas as pd
import handcalcs.render

## 1 - Background

This document encapsulates the work towards the development of the second iteration of the turbine stage of the "Mermaidman and Barnacle Boy" turbopump being developed by the Open Turbopumps amature group.

This document highlights the high-level fluidic design and analyiss conducted for the turbine.

## 2 - High-Level Specifications

Based on the previous studies performed, the following key inputs have been used on the high-level turbine design:

| Parameter        | Value      | Justification                                                                                                       |
| ---------------- | ---------- | ------------------------------------------------------------------------------------------------------------------- |
| Shaft Speed      | 30,000 RPM | Based on the preliminary rotor dynamic analysis, advancing the shaft speed to 30,000 RPM was identified as feasible |
| Pump Power      | 4 kW       | This is the most up-to date pump power requirement                                                                  |

The following inputs from the gas-generator study have additionally been provided:

| Parameter                      | Value       | Justification                                                |
| ------------------------------ | ----------- | ------------------------------------------------------------ |
| Nominal Mixture Ratio          | 1           | Based on Sensitivity Study Conducted on Gas Generator Design |
| Inlet Pressure ($P_o$)         | 20e5        | Based on Sensitivity Study Conducted on Gas Generator Design |
| Inlet Temperature ($T_o$)      | 858 K       | Based on Sensitivity Study Conducted on Gas Generator Design |
| Specific Heat Capacity ($C_p$) | 8575 J/kg K | Based on Sensitivity Study Conducted on Gas Generator Design |
| Specific Heat Ratio ($\gamma$) | 1.164       | Based on Sensitivity Study Conducted on Gas Generator Design |
| Specific Gas Constant (R)      | 518.949 | Based on Sensitivity Study Conducted on Gas Generator Design |



For the sake of consistency, we will parameterise these inputs in an array as was previously shown.

In [21]:
dic = {
    "P_cc": 2500000.0,
    "MR": np.float64(1.0),
    "T_o": np.float64(869.3134455786856),
    "Cp": np.float64(8470.21517719776),
    "gamma": np.float64(1.1651672213813036),
    "R": np.float64(518.9497005130706),
    "ox_stiffness": 1.0,
    "fu_stiffness": 1.0,
    "m_dot_t": 0.1,
    "m_dot_o": np.float64(0.05),
    "m_dot_f": np.float64(0.05),
}

As we only have the power requirements for the pump, we must now derive the power requirements for the shaft, when considering the losses via the bearings and shaft seal.

## 3 - Power Requirements Considering Mechanical Losses

In the shaft design, we have two bearings being used, along with a mechanical seal.

Inorder to determine the losses, we will use `SKF` to estimate the resitive torques expected for both of these bearings.

To do so, we need a rough estimation of what the forces carried by the bearings will be.

A calculation was conducted in the `bearings.ipynb` document for the expected mechanical losses for the system, which was evaluated at approximately 1.6 kW.

This brings the total power requirement for the turbine to 5.6 kW.

We can take an additional 25% Margin against this power requirement, to derive a required turbine power of **7 kW**

## 4 - Turbine Fluidic Design with Geometry Specification

Based on these inputs, we can derive the required turbine power accordingly - using our refined loss model.

We will iterate through a range of u/c_is values to see how our exit conditions vary and how the specific power is like.

In [234]:
R = dic["Cp"] * (dic["gamma"] - 1) / dic["gamma"]

fluid = IdealGas(p=dic["P_cc"], t=dic["T_o"], gamma=dic["gamma"], cp=dic["Cp"], R = R)


alpha_i = 70

stage = TurbineStageDesign(gas=fluid, m_dot=28e-3, omega=25000, alpha=(90 - alpha_i))

stage.set_operating_point(u_cis=0.1, Rt=9, b=15e-3, t=5e-3, delta_r=2e-3, N=6)

result = stage.solve_performance()

Current Error: 11.95386133278994 %
Current Error: 0.8301263990596084 %
Current Error: 0.05157849514192885 %
Current Error: 0.0032281942995786837 %


We can now get an idea of the specific power of the turbine and the associated performance Metrics

In [235]:
performance_dic = result["performance"]
performance_dic = {k: [v] for k, v in performance_dic.items()}
performance_df = pd.DataFrame(performance_dic)

pressure_dic = result["pressure"]
pressure_dic = {k: [v] for k, v in pressure_dic.items()}
pressure_df = pd.DataFrame(pressure_dic)

velocity_dic = result["velocity"]
velocity_dic = {k: [v] for k, v in velocity_dic.items()}
velocity_df = pd.DataFrame(velocity_dic)

temperature_dic = result["temperature"]
temperature_dic = {k: [v] for k, v in temperature_dic.items()}
temperature_df = pd.DataFrame(temperature_dic)

geometry_dic = result["geometry"]
geometry_dic = {k: [v] for k, v in geometry_dic.items()}
geometry_df = pd.DataFrame(geometry_dic)

mach_dic = result["mach"]
mach_dic = {k: [v] for k, v in mach_dic.items()}
mach_df = pd.DataFrame(mach_dic)

angles_dic = result["angles"]
angles_dic = {k: [v] for k, v in angles_dic.items()}
angles_df = pd.DataFrame(angles_dic)

**Performance**

In [236]:
performance_df["Power (kW)"] = performance_df["Power"].multiply(1e-3)
performance_df["Torque (Nm)"] = performance_df["Power"] / (25000 * 2 *np.pi/60)

performance_df

,dh,eps,phi_r,phi_l,m_leakage,eta_l,phi,eta_h,zeta_eps,eta_o,Power,Power (kW),Torque (Nm)
0,1.970612e+06,0.088157,0.693719,0.373641,0.010462,0.626359,0.93,0.263993,0.010091,0.155263,8566.964135,8.566964,3.272339


As can be seen, we are getting an efficiency on the order of **45%** with a specific power of 987 kW/kg/s

This works out to a GG size of the following:

**Pressures**

In [237]:
pressure_df * 1e-5

,p_0,p_1,p_1o,p_1o_r,p_2o_r,p_2o
0,25.0,2.777778,17.795195,12.945591,5.587996,4.580923


**Temperatures**

In [238]:
temperature_df

,t_0,t_1,t_1o_r,t_2,t_2o
0,869.313446,668.092814,830.976407,752.589262,807.894918


**Geometry**

In [239]:
geometry_df

,D_m,A_1,s_c,s_b,D_hub,D_tip,AR
0,0.151662,0.000044,0.003049,0.005549,0.148888,0.154436,0.369904


**Mach**

In [240]:
mach_df

,m_star_c1,m_star_w1,m_star_w2,m_star_c2
0,1.741935,1.60298,1.112018,0.91323


**Angles**

In [241]:
90 -angles_df

,beta_1,beta_2,alpha_2
0,67.657336,89.653979,62.931618


**Velocities**

In [242]:
velocity_df

,u,c_1s,c_1,w_1,a_star_2,w_2,c_2,a_star_3
0,198.525146,1985.251464,1846.283862,1661.119552,1036.269643,1152.350628,967.936782,1021.776399


## 5 - Profile Design

## 6 - Conclusion

The following key Parameters have been derived from this study:

| Parameter                | Value        |
| ------------------------ | ------------ |
| Gas Generator Mass Flow  | 28 g/s       |
| Shaft Speed              | 25,000 rpm   |
| Nozzle outlet Area       | 0.000044 m^2 |
| Nozzle Throat (GG Param) | 0.000011 m^2             |
|                          |              |